<center>
<h1 style="font-family: 'Times New Roman', Times, serif; font-size: 60px;">DAA</h1>
<h2 style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">XGBoost</h2>
</center>

<font face="Times New Roman">

<h3 style="font-family: 'Times New Roman'">Imports</h3>

In [2]:
# Import necessary libraries
import pandas as pd                                                                                 # For data manipulation and analysis
import matplotlib.pyplot as plt                                                                     # For data visualization
import seaborn as sns                                                                               # For enhanced data visualization
from sklearn.model_selection import train_test_split                                                # For splitting data
import xgboost as xgb                                                                               # For the XGBoost
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer    # For evaluating model performance
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV                # For cross-validation
import os                                                                                           # For interacting with the operating system
from sklearn.preprocessing import LabelEncoder

<h3 style="font-family: 'Times New Roman'">Load All Datasets</h3>

In [2]:
final = pd.read_csv("Ficheiros/Ficheiros_Models/Final.csv", na_filter = False)
final_teste = pd.read_csv("Ficheiros/Ficheiros_Models/Final_Teste.csv")

<h3 style="font-family: 'Times New Roman'">XGBoost</h3>

<h4 style="font-family: 'Times New Roman'">Test Locally</h4>

In [3]:
X = final.drop(columns=['injection'])
y = final['injection']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024)

param_grid = {
    'learning_rate': [0.085, 0.09, 0.095, 0.1],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7, 10],
    'objective': ['binary:logistic', 'multi:softmax', 'multi:softprob'],
}

modelo = xgb.XGBClassifier(random_state=2024) # objective='binary:logistic'
scoring = make_scorer(accuracy_score)
cv_value = 10
grid_accuracy = GridSearchCV(estimator=modelo, param_grid=param_grid, scoring=scoring, cv=cv_value, refit=True)

grid_accuracy.fit(X_train, y_train)

best_model = grid_accuracy.best_estimator_
best_model_params = best_model.get_params()

print("Hyperparameters of the Best Model:")
for param, value in best_model_params.items():
    print(f"{param}: {value}")

y_pred = best_model.predict(X_test)

# Print the cross-validated accuracy scores
cv_scores = grid_accuracy.cv_results_['mean_test_score']
# print("Cross-validated Accuracy Scores: {}".format(cv_scores))
print("Mean Accuracy: {:.2f}%".format(cv_scores.mean() * 100))

accuracy_injection = accuracy_score(y_test, y_pred)
print("XGBoost Model Accuracy: {:.2f}%".format(accuracy_injection * 100))

classification_report_str = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report_str)

cm = confusion_matrix(y_test, y_pred)

# Define a custom color palette
color_palette = sns.light_palette("seagreen", as_cmap=True)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap=color_palette, cbar=False)

# Add labels and title
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Customize tick labels
tick_labels = ['None', 'Low', 'Medium', 'High', 'Very High']
plt.xticks(ticks=range(len(tick_labels)), labels=tick_labels)
plt.yticks(ticks=range(len(tick_labels)), labels=tick_labels)

plt.show()

<h4 style="font-family: 'Times New Roman'">Kaggle</h4>

In [3]:
final_kaggle = pd.read_csv("Ficheiros/Ficheiros_Models/Final.csv", na_filter = False)
final_teste_kaggle = pd.read_csv("Ficheiros/Ficheiros_Models/Final_Teste.csv")

In [4]:
X_kaggle = final_kaggle.drop(columns=['injection'])
y_kaggle = final_kaggle['injection']

In [5]:
param_grid_kaggle = {
    'learning_rate': [0.09],
    'n_estimators': [150],
    'max_depth': [5],
    'objective': ['multi:softprob'],
}

modelo_kaggle = xgb.XGBClassifier(random_state=2023) # objective='binary:logistic', 
scoring_kaggle = make_scorer(accuracy_score)
cv_value_kaggle = 10
grid_accuracy_kaggle = GridSearchCV(estimator=modelo_kaggle, param_grid=param_grid_kaggle, scoring=scoring_kaggle, cv=cv_value_kaggle, refit=True)

grid_accuracy_kaggle.fit(X_kaggle, y_kaggle)

best_model_kaggle = grid_accuracy_kaggle.best_estimator_
X_teste_kaggle = final_teste
y_pred_kaggle = best_model_kaggle.predict(X_teste_kaggle)

c:\Users\afons\miniconda3\envs\daa1\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\afons\miniconda3\envs\daa1\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\afons\miniconda3\envs\daa1\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\afons\miniconda3\envs\daa1\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\afons\miniconda3\envs\daa1\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.

NameError: name 'final_teste' is not defined

In [ ]:
reverse_mapping = {0: 'None', 1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}

# Aplica o mapeamento inverso às previsões 'y_pred'
y_pred_kaggle = [reverse_mapping[pred] for pred in y_pred_kaggle]

In [ ]:
# Salvar as previsões em 'prediction_results.csv'
with open('Outputs/XGBoost.csv', 'w') as file:
    file.write("RowId,Result\n")
    for row_id, prediction in enumerate(y_pred_kaggle, start=1):
        file.write(f"{row_id},{prediction}\n")

print("Previsões salvas em XGBoost.csv")

Previsões salvas em XGBoost.csv
